In [1]:
import torch
import sys
sys.path.append('/home/zjy/project/MetaIM')
pwd = '/home/zjy/project/MetaIM/data'
device = torch.device('cuda:4' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=4)

In [2]:
from torch_geometric.datasets import Planetoid

cora_dataset = Planetoid(root=pwd+'/cora', name='cora')
data = cora_dataset[0]
edge_index = data.edge_index

In [3]:
import numpy as np
individual_infection_path = pwd+'/for_meta/cora_individual_infection_sir_200.npy'
seeds_infection_path = pwd+'/for_meta/cora_seed_infection_sir_200_sample_1000.npy'

individual_infection = np.load(individual_infection_path)
seeds_infection = np.load(seeds_infection_path)
individual_infection.shape,seeds_infection.shape

((2708, 2708), (1000, 2, 2708))

In [4]:
import torch
from torch_geometric.utils import to_scipy_sparse_matrix
import scipy.sparse as sp

# 转换为 scipy 稀疏矩阵
adj = to_scipy_sparse_matrix(edge_index)

adj = torch.Tensor(adj.toarray()).to_sparse()
adj


tensor(indices=tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
                       [ 633, 1862, 2582,  ...,  598, 1473, 2706]]),
       values=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
       size=(2708, 2708), nnz=10556, layout=torch.sparse_coo)

In [5]:
def graph_laplacian_embedding(adjacency_matrix, num_dimensions):
    # 将邻接矩阵转换为PyTorch张量
    adjacency_matrix = torch.tensor(adjacency_matrix, dtype=torch.float32)
    
    # Step 1: 计算度矩阵
    degree_matrix = torch.diag(adjacency_matrix.sum(dim=1))
    
    # Step 2: 计算拉普拉斯矩阵
    laplacian_matrix = degree_matrix - adjacency_matrix
    
    # Step 3: 计算拉普拉斯矩阵的特征向量和特征值
    eigenvalues, eigenvectors = torch.linalg.eigh(laplacian_matrix)
    
    # Step 4: 选择低维表示
    # 取特征值非零的前 num_dimensions 个特征向量作为嵌入结果
    embedding = eigenvectors[:, :num_dimensions]
    
    return embedding

def edge_index_to_adj(edge_index, num_nodes):
    adj = torch.zeros((num_nodes, num_nodes))
    for i in range(edge_index.size(1)):
        start_node = edge_index[0, i]
        end_node = edge_index[1, i]
        adj[start_node, end_node] = 1
        adj[end_node, start_node] = 1  # 如果是有向图，可以不需要这行
    return adj

latent_dim = 64

adj_n = edge_index_to_adj(edge_index, individual_infection.shape[0])

laplace_emb = graph_laplacian_embedding(adj_n, latent_dim)

laplace_emb = laplace_emb.to(device)

/tmp/ipykernel_2534612/1199588063.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  adjacency_matrix = torch.tensor(adjacency_matrix, dtype=torch.float32)


In [6]:
seed_num = int(seeds_infection[0][0].sum())

In [7]:
from torch.utils.data import Dataset, DataLoader, random_split


class CustomDataset(Dataset):
    def __init__(self, individual_infection,seeds_infection, seed_num):
        self.individual_infection = individual_infection
        self.seeds_infection = seeds_infection
        self.feat_shape = (len(individual_infection), seed_num)

    def __len__(self):
        return len(self.seeds_infection)

    def __getitem__(self, idx):
        seeds= np.nonzero(self.seeds_infection[idx][0])[0]
        
        feature = torch.zeros(self.feat_shape[0],self.feat_shape[1])
        for i in range(len(seeds)):
            seed_i_infection = torch.tensor(self.individual_infection[seeds[i]])
            feature[:, i] = seed_i_infection
            
        label = self.seeds_infection[idx][1]
        
        return self.seeds_infection[idx][0], feature, label

dataset = CustomDataset(individual_infection, seeds_infection, seed_num)

In [8]:
# 定义划分比例
train_ratio = 0.8
test_ratio = 0.2

# 划分数据集
train_dataset, test_dataset = random_split(dataset, [int(len(dataset)*train_ratio), int(len(dataset)*test_ratio)])

train_batch_size = 64
test_batch_size = 4

# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True, drop_last=False)
test_loader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)

In [9]:
import torch
import torch.nn as nn

from torch_geometric.nn import GATConv
from torch.optim import Adam, SGD
import torch.nn.functional as F

class GAT(nn.Module):
    def __init__(self, feat_dim, latent_dim, hidden_channels, out_channels, num_heads):
        super(GAT, self).__init__()
        # self.linear1 = nn.Linear(latent_dim, 128)
        self.conv1 = GATConv(feat_dim, hidden_channels, heads=num_heads)
        self.conv2 = GATConv(hidden_channels * num_heads, hidden_channels * num_heads, heads=1)
        self.linear2 = nn.Linear(hidden_channels * num_heads+latent_dim, 1)

    def forward(self, feat_i, x_i, edge_index):
        x = self.conv1(feat_i, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv2(x, edge_index)
        
        # latten = F.relu(self.linear1(x_i))
        x =  torch.cat((x, x_i), dim=-1)
        x = self.linear2(x)
        return F.relu(x)

In [10]:
feat_num = seed_num

forward_model = GAT(feat_num,latent_dim, 512, 1, 4)

optimizer = Adam([{'params': forward_model.parameters()}], 
                 lr=0.0001)

adj = adj.to(device)
forward_model = forward_model.to(device)
forward_model.train()

GAT(
  (conv1): GATConv(135, 512, heads=4)
  (conv2): GATConv(2048, 2048, heads=1)
  (linear2): Linear(in_features=2112, out_features=1, bias=True)
)

In [11]:
def normalize_features(tensor):
    # 计算每个特征的最小值和最大值
    min_vals, _ = torch.min(tensor, dim=0)
    max_vals, _ = torch.max(tensor, dim=0)

    # 对每个特征进行归一化
    normalized_tensor = (tensor - min_vals) / (max_vals - min_vals)
    return normalized_tensor

In [12]:
edge_index = edge_index.to(device)
top_num = 500
# encoder = vae_model.Encoder

for epoch in range(2000):

    total_overall = 0
    forward_loss = 0

    mean_train_accuracy = 0
    for batch_idx, x_feature_label in enumerate(train_loader):  
        x =  x_feature_label[0].to(device)     
        features = x_feature_label[1].to(device)
        labels = x_feature_label[2].to(device)
        optimizer.zero_grad()
        
        loss = 0
        train_accuracy = 0
        for i, feat_i in enumerate(features):
            
            # log_sum = torch.sum(torch.log(1 - x_i), dim=1, keepdim=True)
            # feat = 1 - torch.exp(log_sum)
            x_i = x[i]
            x_i = torch.matmul(x_i, laplace_emb)
            
            
            # x_i = encoder(x_i).detach()
            # x_i = torch.rand(latent_dim).to(device)
            # x_i = normalize_features(x_i)
            x_i = x_i.expand(features.shape[1], -1)
            # final_feat_i =  torch.cat((feat_i, x_i), dim=1)
            
            
            y_i = labels[i]
            y_hat = forward_model(feat_i, x_i, edge_index)
            _, top_indices_true = torch.topk(y_i.clone(), top_num)
            label_2 = torch.zeros(y_i.shape).to(device)
            label_2[top_indices_true] = 1
            
            _, top_indices_predict = torch.topk(y_hat.clone().squeeze(-1), top_num)
            
            # 将张量数组转换为Python列表
            list1 = top_indices_true.tolist()
            list_pre = top_indices_predict.tolist()

            # 使用集合操作找到交集
            intersection = list(set(list1) & set(list_pre))
            accuracy_i = len(intersection) / top_num       
            train_accuracy += accuracy_i 

            forward_loss = 0.5*F.mse_loss(y_hat.squeeze(-1), y_i, reduction='sum') + F.mse_loss(y_hat.squeeze(-1), label_2, reduction='sum')    
            loss += forward_loss    
        
        total_overall += loss.item()    
        train_accuracy /= len(features)
        mean_train_accuracy = train_accuracy
        loss = loss/features.size(0)
        loss.backward()
        optimizer.step()
        # for p in forward_model.parameters():
        #     p.data.clamp_(min=0)
        
        
    print("Epoch: {}".format(epoch+1), 
        "\tTotal: {:.4f}".format(total_overall / train_batch_size),
        "\tMean_train_accuracy: {:.4f}".format(mean_train_accuracy),
        )  
    
    mean_accuracy = 0
    mean_accuracy_sum = 0

    
    for batch_idx, x_feature_label in enumerate(test_loader):  
        x =  x_feature_label[0].to(device)
        features = x_feature_label[1].to(device)
        labels = x_feature_label[2].to(device)
        
        accuracy = 0
        accuracy_sum = 0
        
        for i, feat_i in enumerate(features):
            # x_i = x[i]
            # x_i = encoder(x_i).detach()
            # x_i = normalize_features(x_i)
            x_i = x[i]
            x_i = torch.matmul(x_i, laplace_emb)
            x_i = x_i.expand(features.shape[1], -1)
            # final_feat_i =  torch.cat((feat_i, x_i), dim=1)
            y_i = labels[i]
            _, top_indices_true = torch.topk(y_i, top_num)
            
            y_hat = forward_model(feat_i, x_i, edge_index)
            
            _, top_indices_predict = torch.topk(y_hat.squeeze(-1), top_num)
            
            sum_pre = torch.sum(feat_i, dim=1, keepdim=True)
            _, top_indices_sum = torch.topk(sum_pre.squeeze(-1), top_num)
            
            # 将张量数组转换为Python列表
            list1 = top_indices_true.tolist()
            list_pre = top_indices_predict.tolist()
            
            list_sum = top_indices_sum.tolist()

            # 使用集合操作找到交集
            intersection = list(set(list1) & set(list_pre))
            
            intersection_sum = list(set(list1) & set(list_sum))
            
            accuracy_i = len(intersection) / top_num       
            accuracy += accuracy_i 
            accuracy_sum += len(intersection_sum) / top_num  
        accuracy /= test_batch_size
        accuracy_sum/= test_batch_size
        mean_accuracy = accuracy
        mean_accuracy_sum = accuracy_sum
        break
    
    print(
        "\tMean_test_accuracy: {:.4f}".format(mean_accuracy),
        "\tMean_test_accuracy_sum: {:.4f}".format(mean_accuracy_sum)
        )  

    

Epoch: 1 	Total: 5783.3029 	Mean_train_accuracy: 0.4468
	Mean_test_accuracy: 0.4675 	Mean_test_accuracy_sum: 0.5190
Epoch: 2 	Total: 4960.4995 	Mean_train_accuracy: 0.5100
	Mean_test_accuracy: 0.5250 	Mean_test_accuracy_sum: 0.5190
Epoch: 3 	Total: 4655.7780 	Mean_train_accuracy: 0.5502
	Mean_test_accuracy: 0.5530 	Mean_test_accuracy_sum: 0.5190
Epoch: 4 	Total: 4456.5277 	Mean_train_accuracy: 0.5623
	Mean_test_accuracy: 0.5770 	Mean_test_accuracy_sum: 0.5190
Epoch: 5 	Total: 4353.5906 	Mean_train_accuracy: 0.5763
	Mean_test_accuracy: 0.5940 	Mean_test_accuracy_sum: 0.5190
Epoch: 6 	Total: 4304.5269 	Mean_train_accuracy: 0.5776
	Mean_test_accuracy: 0.5945 	Mean_test_accuracy_sum: 0.5190
Epoch: 7 	Total: 4269.3248 	Mean_train_accuracy: 0.5838
	Mean_test_accuracy: 0.6015 	Mean_test_accuracy_sum: 0.5190
Epoch: 8 	Total: 4239.8947 	Mean_train_accuracy: 0.5904
	Mean_test_accuracy: 0.6070 	Mean_test_accuracy_sum: 0.5190
Epoch: 9 	Total: 4208.0143 	Mean_train_accuracy: 0.6012
	Mean_test_accur